In [1]:
# imports
import requests
import json
import os
import time
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

In [2]:
# let's speed up the scraping proccess by scraping multiple urls in one batch

# generic url scraper

def url_scraper_batch(
                urlbase = '',
                urls = [],
                selenium = True,
                windowSize = "1280,720",
                headless = True,
                quitOnEnd = True,
                waitForElement = '',
                waitForId = '',
                waitForClass = '',
                waitForIframeById = '',
                pressLink = [],
                waitBetweenPress = 30,
                waitUntilTimeout = 20,
               ):
    #meta = {
    #    'url': url
    #}
    
    # simple version
    if selenium == False:
        #page = requests.get(url)
        #meta['status'] = page.status_code
        #html = page.content
        print('batched url is for selenium only')
        return ''
      
    # selenium version
    if selenium == True:
        CHROMEDRIVER_PATH = './.selenium/chromedriver'
        WINDOW_SIZE = windowSize

        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

        driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
        driver.implicitly_wait(waitBetweenPress)
        
        result = []
        for url in urls:
            item = {
                'url': url
            }
            
            driver.get(urlbase + url)

            # wait for a given tag to be loaded (javascript generated code)
            if waitForElement != '':
                try:
                    element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.TAG_NAME, waitForElement)))
                    print ("Page is ready!", urlbase + url)
                except TimeoutException:
                    print ("element - loading took too much time!", urlbase + url)

            # wait for a given ID to be loaded (javascript generated code)
            if waitForId != '':
                try:
                    element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.ID, waitForId)))
                    print ("Page is ready!", urlbase + url)
                except TimeoutException:
                    print ("id - loading took too much time!", urlbase + url)

            # wait for a given class to be loaded (javascript generated code)
            if waitForClass != '':
                try:
                    # not-found
                    element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.CLASS_NAME, waitForClass)))
                    print ("Page is ready!", urlbase + url)
                except TimeoutException:
                    print ("class - loading took too much time!", urlbase + url)
                    
            # wait for a given ID to be loaded (javascript generated code)
            if waitForIframeById != '':
                try:
                    element = WebDriverWait(driver, waitUntilTimeout).until(EC.presence_of_element_located((By.ID, waitForIframeById)))
                    
                    driver.switch_to.frame(element)
                    
                    item['iframe'] = driver.page_source
                    
                    driver.switch_to.default_content()
                    
                    print ("Page is ready!", urlbase + url)
                except TimeoutException:
                    print ("iframe - loading took too much time!", urlbase + url)

            # press link
            if len(pressLink)>0:
                driver.implicitly_wait(waitBetweenPress)
                try:
                    for link in pressLink:
                        print(link)
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.LINK_TEXT, link)))
                        element.click()
                except TimeoutException:
                    print ("link - loading took too much time!", urlbase + url)

            item['html'] = driver.page_source
            result.append(item)
            
            
        if quitOnEnd:
            driver.close()
    
    return result

urlbase = 'https://www.kaggle.com'
urls = [
    '/austinreese/craigslist-carstrucks-data',
    '/google/tinyquickdraw',
    '/babyoda/women-entrepreneurship-and-labor-force',
]
urls2 = [
    'https://www.kaggle.com/asimandia/lyft3d-inference-kernel/notebook',
    'https://www.kaggle.com/opanichev/lightgbm-and-simple-features',
]

#result = url_scraper_batch(urlbase, urls, waitForClass='content-box', headless = False, quitOnEnd = True)
result = url_scraper_batch('', urls2, waitForIframeById='rendered-kernel-content', headless = False, quitOnEnd = True)
#print(result)
for item in result:
    print(item['url'], len(item['html']))
    #print(len(item['iframe']))

Page is ready! https://www.kaggle.com/asimandia/lyft3d-inference-kernel/notebook
iframe - loading took too much time! https://www.kaggle.com/opanichev/lightgbm-and-simple-features
https://www.kaggle.com/asimandia/lyft3d-inference-kernel/notebook 465349
https://www.kaggle.com/opanichev/lightgbm-and-simple-features 3221824


In [3]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
print(store_data('Hello', '../data/repositories/mlart/test.txt'))
print(load_data('../data/repositories/mlart/test.txt'))

# test json
print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
print(load_data('../data/repositories/mlart/test.json', fromJson=True))

#store_data(result[0]['html'], '../data/repositories/kaggle/notebook.html')
#store_data(result[0]['iframe'], '../data/repositories/kaggle/kernel.html')

5
Hello
22
{'msg': 'Hello World'}


In [4]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [ ]:
# get all sites from csv and store them for later feature scraping

# competitions
#csv = '../data/repositories/kaggle/kaggle_competitions.csv'
#folder = '../data/repositories/kaggle/competitions'

# datasets
csv = '../data/repositories/kaggle/kaggle_datasets.csv'
folder = '../data/repositories/kaggle/datasets'

df = pd.read_csv(csv)
df_links = list(df['link'])
batch_size = 25

chunks = [df_links[i:i + batch_size] for i in range(0, len(df_links), batch_size)]
chunks_len = len(chunks)
print('total/chunks', len(df_links), len(chunks))

for i, chunk in enumerate(chunks):
    print('### chunk', i, '/', chunks_len, '###')
    
    # check if chunk is already scraped
    chunk_datasets = []
    chunk_notebooks = []
    for j, link in enumerate(chunk):
        print('# chunk', i, '/', chunks_len, '# item ', j, link)

        temp = link.split('/')
        author = temp[1]
        name = temp[2]
        name = name.replace(' ','_').replace(':','')
        #print(author, name)
    
        # check datasets
        file_d = os.path.join(folder, author, name+'/dataset.html')
        if not os.path.isfile(file_d):
            #print('dataset.html not found', file)
            chunk_datasets.append(link)
            
            # create folders
            create_folder(os.path.join(folder, author+'/'))
            create_folder(os.path.join(folder, author, name+'/'))
        #else:
        #    print('dataset.html already exists')
            
        # check if notebooks.html exists
        file_n = os.path.join(folder, author, name+'/notebooks.html')
        if not os.path.isfile(file_n):
            
            # before adding the link, check if there are notebooks attached
            html = load_data(file_d)
            soup = BeautifulSoup(html, 'html.parser')
            notebooks = soup.select('span.pageheader__link-count > span')
            if len(notebooks) > 0:
            
                #print('notebooks.html not found', file)
                chunk_notebooks.append(link+'/notebooks?sortBy=voteCount')

        
    
    #print(len(chunk_output), chunk_output)
    
    # get content for chunk (datasets)
    if len(chunk_datasets) > 0:
        result = url_scraper_batch(urlbase, chunk_datasets, waitForClass='content-box', headless = True, quitOnEnd = True)
    
        for item in result:
            print(item['url'], len(item['html']))
            #file = os.path.join(folder, item['url']) + '/dataset.html'
            file = folder + item['url'] + '/dataset.html'
            file = file.replace(' ','_').replace(':','')
            print(file)
            store_data(item['html'], file)
    
    # get content for chunk (notebooks)
    if len(chunk_notebooks) > 0:
        result = url_scraper_batch(urlbase, chunk_notebooks, waitForClass='sc-qcrOD', headless = True, quitOnEnd = True)
        
        for item in result:
            print(item['url'], len(item['html']))
            #file = os.path.join(folder, item['url']) + '/notebooks.html'
            file = folder + item['url'] + '.html'
            file = file.replace(' ','_').replace(':','').replace('?sortBy=voteCount','')
            print(file)
            store_data(item['html'], file)
    
        #break
    



In [ ]:
# scrape links from notebooks.html

folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
file = 'notebooks.html'
out = 'notebooks.json'
url = 'https://www.kaggle.com/c/3d-object-detection-for-autonomous-vehicles/notebooks?sortBy=voteCount'

def scrape_notebook_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []
    
    partial = soup.find('div', class_="km-list km-list--avatar-list km-list--three-line")
    if partial == None:
        return []
    items = partial.find_all('li', {"role": "listitem"})
    #print(len(items))
    
    for i, item in enumerate(items):
        #print(i, item)
        try:
            link = item.select('a.sc-qcrOD')[0].get('href')
            result.append(link)
        except:
            print('an error occured')
            break
        
        #print(link)
    
    return result

html = load_data(folder_base+folder+file)
#result = url_scraper_batch('', [url], waitForClass='sc-qcrOD', headless = False, quitOnEnd = True)
#html = result[0]['html']
links = scrape_notebook_links(html)
print(links)
#store_data(links, folder_base+folder+out, toJson=True)

In [ ]:
# parse notebooks.html to notebooks.json (links)

#path = '../data/repositories/kaggle/competitions/'
path = '../data/repositories/kaggle/datasets/'
file = 'notebooks.html'
folders = os.listdir(path)
for folder in folders:
    subfolders = os.listdir(os.path.join(path,folder))
    for subfolder in subfolders:
        item = os.path.join(path,folder,subfolder,file)
        print(item)
        if os.path.isfile(item):
            html = load_data(item)
            links = scrape_notebook_links(html)
            print('link', links)
            if len(links) > 0 and not 'titanic' in link.lower():
                output = item.replace('.html','.json')
                print('output', output)
                store_data(links, output, toJson=True)

In [ ]:
# download notebooks

#path = '../data/repositories/kaggle/competitions/'
path = '../data/repositories/kaggle/datasets/'
file = 'notebooks.json'
storefolder = 'notebooks/'
urlbase = 'https://www.kaggle.com'
notebook = 'notebook_02.html'
kernel = 'kernel.html'

folders = os.listdir(path)
print(len(folders))
i = 0

for folder in folders:
    subfolders = os.listdir(os.path.join(path,folder))
    for subfolder in subfolders:
        item = os.path.join(path,folder,subfolder,file)
        i+=1

        print('###', i, item)
        if os.path.isfile(item):
            links = load_data(item, fromJson=True)
            #print(len(links), links)

            chunk = []
            for j, link in enumerate(links):
                #print(j, link)

                temp = link.split('/')
                author = temp[1]
                name = temp[2]
                name = name.replace(' ','_').replace(':','')

                # create folders
                f1 = os.path.join(path, folder, subfolder, storefolder, author+'/')
                f2 = os.path.join(path, folder, subfolder, storefolder, author, name+'/')
                create_folder(f1)
                create_folder(f2)

                # check notebooks
                fp = os.path.join(f2, notebook)
                #print(fp)
                #break
                if not os.path.isfile(fp) and not 'kerneler' in fp:
                    #print('notebooks.html not found', fp)
                    chunk.append(link)

            print('### chunk:', i, 'len:', len(chunk), 'links:', chunk)

            # get content for chunk (datasets)
            if len(chunk) > 0:
                try:
                    result = url_scraper_batch(urlbase, chunk, waitForIframeById='rendered-kernel-content', headless = True)

                    for item in result:
                        # store notebook
                        #print(item['url'], len(item['html']))
                        fp = path + '/' + folder + '/' + subfolder + '/' + storefolder + '/' + item['url'] + '/' + notebook
                        fp = fp.replace(' ','_').replace(':','').replace('//','/').replace('//','/')
                        print('### output file', fp)
                        store_data(item['html'], fp)
                        
                        # store kernel
                        if 'iframe' in item:
                            fp = fp.replace(notebook, kernel)
                            print('### output file', fp)
                            store_data(item['iframe'], fp)
                except Exception as e:
                    print("Oops!", e.__class__, "occurred.")
                    print(e)
                    #break
                   
            quit = 100000
            if i > quit:
                break


8467
### 1 ../data/repositories/kaggle/datasets/4quant\depth-generation-lightfield-imaging\notebooks.json
### chunk: 1 len: 0 links: []
### 2 ../data/repositories/kaggle/datasets/4quant\eye-gaze\notebooks.json
### chunk: 2 len: 0 links: []
### 3 ../data/repositories/kaggle/datasets/4quant\simplefoam\notebooks.json
### chunk: 3 len: 0 links: []
### 4 ../data/repositories/kaggle/datasets/4quant\soft-tissue-sarcoma\notebooks.json
### chunk: 4 len: 0 links: []
### 5 ../data/repositories/kaggle/datasets/a11rand0m\catdoginfo\notebooks.json
### chunk: 5 len: 0 links: []
### 6 ../data/repositories/kaggle/datasets/a13x10\basic-arabic-vocal-emotions-dataset\notebooks.json
### chunk: 6 len: 0 links: []
### 7 ../data/repositories/kaggle/datasets/a1997425\roberta\notebooks.json
### chunk: 7 len: 0 links: []
### 8 ../data/repositories/kaggle/datasets/a2015003713\militaryaircraftdetectiondataset\notebooks.json
### 9 ../data/repositories/kaggle/datasets/a24998667\bert001\notebooks.json
### chunk: 9 le

### chunk: 72 len: 0 links: []
### 73 ../data/repositories/kaggle/datasets/abbrivia\names-from-35k-wikipedia-movie-plots\notebooks.json
### chunk: 73 len: 0 links: []
### 74 ../data/repositories/kaggle/datasets/abbrivia\septuagint\notebooks.json
### chunk: 74 len: 0 links: []
### 75 ../data/repositories/kaggle/datasets/abbrivia\septuagintnames\notebooks.json
### 76 ../data/repositories/kaggle/datasets/abcsds\highest-mountains\notebooks.json
### 77 ../data/repositories/kaggle/datasets/abcsds\pokemon\notebooks.json
### chunk: 77 len: 0 links: []
### 78 ../data/repositories/kaggle/datasets/abcsds\pokemongo\notebooks.json
### chunk: 78 len: 0 links: []
### 79 ../data/repositories/kaggle/datasets/abdallahalidev\plantvillage-dataset\notebooks.json
### chunk: 79 len: 0 links: []
### 80 ../data/repositories/kaggle/datasets/abdelaliezzyn\custom-coco\notebooks.json
### chunk: 80 len: 0 links: []
### 81 ../data/repositories/kaggle/datasets/abdelaliezzyn\humanscoco\notebooks.json
### chunk: 81 len

### chunk: 163 len: 0 links: []
### 164 ../data/repositories/kaggle/datasets/abhishek\bert-base-uncased\notebooks.json
### chunk: 164 len: 0 links: []
### 165 ../data/repositories/kaggle/datasets/abhishek\bertsrc\notebooks.json
### chunk: 165 len: 0 links: []
### 166 ../data/repositories/kaggle/datasets/abhishek\efficientnet-pytorch\notebooks.json
### chunk: 166 len: 0 links: []
### 167 ../data/repositories/kaggle/datasets/abhishek\gpt2-pytorch\notebooks.json
### chunk: 167 len: 0 links: []
### 168 ../data/repositories/kaggle/datasets/abhishek\hindi-oscar-corpus\notebooks.json
### chunk: 168 len: 0 links: []
### 169 ../data/repositories/kaggle/datasets/abhishek\leafmodel\notebooks.json
### chunk: 169 len: 0 links: []
### 170 ../data/repositories/kaggle/datasets/abhishek\maskrcnn-utils\notebooks.json
### chunk: 170 len: 0 links: []
### 171 ../data/repositories/kaggle/datasets/abhishek\pretrained-model-weights-pytorch\notebooks.json
### chunk: 171 len: 0 links: []
### 172 ../data/reposit

### 249 ../data/repositories/kaggle/datasets/adamsk\top-booked-hotels\notebooks.json
### chunk: 249 len: 0 links: []
### 250 ../data/repositories/kaggle/datasets/adamyodfat\1001-israeli-pop-dataset\notebooks.json
### chunk: 250 len: 0 links: []
### 251 ../data/repositories/kaggle/datasets/adangalvan\covidmexico\notebooks.json
### chunk: 251 len: 0 links: []
### 252 ../data/repositories/kaggle/datasets/adaoduque\campeonato-brasileiro-de-futebol\notebooks.json
### chunk: 252 len: 0 links: []
### 253 ../data/repositories/kaggle/datasets/adarsh4u\drug-related-deaths\notebooks.json
### chunk: 253 len: 0 links: []
### 254 ../data/repositories/kaggle/datasets/adaubas\ingvchallengefeatures\notebooks.json
### chunk: 254 len: 0 links: []
### 255 ../data/repositories/kaggle/datasets/adbluu\secure-dataset\notebooks.json
### chunk: 255 len: 0 links: []
### 256 ../data/repositories/kaggle/datasets/adcastillogill\world-shapefile\notebooks.json
### chunk: 256 len: 0 links: []
### 257 ../data/repositor

### 377 ../data/repositories/kaggle/datasets/agailloty\house-price-in-canada\notebooks.json
### chunk: 377 len: 0 links: []
### 378 ../data/repositories/kaggle/datasets/agajorte\detroit-daily-temperatures-with-artificial-warming\notebooks.json
### chunk: 378 len: 0 links: []
### 379 ../data/repositories/kaggle/datasets/agajorte\zoo-animals-extended-dataset\notebooks.json
### chunk: 379 len: 0 links: []
### 380 ../data/repositories/kaggle/datasets/agaluzin\yelp-data-challenge\notebooks.json
### 381 ../data/repositories/kaggle/datasets/agalya26\camera-negative\notebooks.json
### 382 ../data/repositories/kaggle/datasets/agasca\retail-sales\notebooks.json
### chunk: 382 len: 0 links: []
### 383 ../data/repositories/kaggle/datasets/agatii\total-sale-2018-yearly-data-of-grocery-shop\notebooks.json
### chunk: 383 len: 0 links: []
### 384 ../data/repositories/kaggle/datasets/agbajeabdullateef\a-database-of-leaf-images-from-mendeley-data\notebooks.json
### chunk: 384 len: 0 links: []
### 385 ..

...

Page is ready! https://www.kaggle.com/blogdish/reading-books-of-nepaul-era
Page is ready! https://www.kaggle.com/apoorvalal/reading-books-of-nepaul-era
### output file ../data/repositories/kaggle/datasets/blogdish/nineteenth-century-works-on-nepal/notebooks/blogdish/reading-books-of-nepaul-era/notebook_02.html
### output file ../data/repositories/kaggle/datasets/blogdish/nineteenth-century-works-on-nepal/notebooks/blogdish/reading-books-of-nepaul-era/kernel.html
### output file ../data/repositories/kaggle/datasets/blogdish/nineteenth-century-works-on-nepal/notebooks/apoorvalal/reading-books-of-nepaul-era/notebook_02.html
### output file ../data/repositories/kaggle/datasets/blogdish/nineteenth-century-works-on-nepal/notebooks/apoorvalal/reading-books-of-nepaul-era/kernel.html
### 2094 ../data/repositories/kaggle/datasets/bloodaxe\xview2-damage-assessment\notebooks.json
### chunk: 2094 len: 1 links: ['/bloodaxe/deep-learning-for-satellite-image-processing']
Page is ready! https://www.kag

Page is ready! https://www.kaggle.com/afiseppe/regression-challenge-day-4-multiple-predictors
Page is ready! https://www.kaggle.com/seroppi/regression-challenge-day-4-the-perfect-model
Page is ready! https://www.kaggle.com/sheshu/shopping-pattern-and-obesity
Page is ready! https://www.kaggle.com/jugalashar/regression-challenge-day-4-5-elasticnet
Page is ready! https://www.kaggle.com/jugalashar/regression-challenge-day-4-b79108
Page is ready! https://www.kaggle.com/deepthoughtdasc/regression-challenge-day-4-gamma-distribution
### output file ../data/repositories/kaggle/datasets/bls/eating-health-module-dataset/notebooks/rtatman/regression-challenge-day-4/notebook_02.html
### output file ../data/repositories/kaggle/datasets/bls/eating-health-module-dataset/notebooks/rtatman/regression-challenge-day-4/kernel.html
### output file ../data/repositories/kaggle/datasets/bls/eating-health-module-dataset/notebooks/rtatman/regression-challenge-day-4-gamma-distribution/notebook_02.html
### output 

Page is ready! https://www.kaggle.com/gautamborg/initial-wage-exploration
Page is ready! https://www.kaggle.com/maddula/first-look
### output file ../data/repositories/kaggle/datasets/bls/wage-estimates/notebooks/gautamborg/initial-wage-exploration/notebook_02.html
### output file ../data/repositories/kaggle/datasets/bls/wage-estimates/notebooks/gautamborg/initial-wage-exploration/kernel.html
### output file ../data/repositories/kaggle/datasets/bls/wage-estimates/notebooks/maddula/first-look/notebook_02.html
### output file ../data/repositories/kaggle/datasets/bls/wage-estimates/notebooks/maddula/first-look/kernel.html
### 2103 ../data/repositories/kaggle/datasets/bluehorseshoe\uk-2016-road-safety-data\notebooks.json
### chunk: 2103 len: 1 links: ['/bluehorseshoe/uk-2016-road-safety-dataset-cycling-prediction']
Page is ready! https://www.kaggle.com/bluehorseshoe/uk-2016-road-safety-dataset-cycling-prediction
### output file ../data/repositories/kaggle/datasets/bluehorseshoe/uk-2016-roa

Page is ready! https://www.kaggle.com/wuyhbb/final-small
Page is ready! https://www.kaggle.com/hidehisaarai1213/imet-pytorch-starter
Page is ready! https://www.kaggle.com/artgor/cassava-disease-identification-with-lightning
Page is ready! https://www.kaggle.com/mahmudds/siim-isic-melanoma-classification
Page is ready! https://www.kaggle.com/seefun/basic-eda-and-pytorch-resnext-model
Page is ready! https://www.kaggle.com/adityakumar01/aptos-resnet34-with-progressive-resizing-fastai
Page is ready! https://www.kaggle.com/naushads/aptos-2019-road-to-victory
Page is ready! https://www.kaggle.com/bibek777/adversarial-validation
Page is ready! https://www.kaggle.com/xooca1/aptos-2
Page is ready! https://www.kaggle.com/itwice/kernel266d1c7f1f
Page is ready! https://www.kaggle.com/hidehisaarai1213/petfinder-lots-of-images-delete-resc-coding2
Page is ready! https://www.kaggle.com/hidehisaarai1213/petfinder-lots-of-images-delete-resc-coding
Page is ready! https://www.kaggle.com/kaerunantoka/extra

Page is ready! https://www.kaggle.com/tonyliu/swordsman-and-their-swords-a-tree-model
Page is ready! https://www.kaggle.com/ryanburge/what-tools-do-academics-use
iframe - loading took too much time! https://www.kaggle.com/rmhogervorst/co-occurance-of-tools
Page is ready! https://www.kaggle.com/hanyan/popular-tools-by-academic-fields2
Page is ready! https://www.kaggle.com/hanyan/tools-by-academic-fields
